In [1]:
import os
import csv
import json
import logging
import math
from typing import List, Dict, Tuple, Callable, Iterable, Optional, Any

import torch
import torch.nn as nn
from sentence_transformers import SentenceTransformer, models, InputExample, losses
from sentence_transformers.evaluation import LabelAccuracyEvaluator, SentenceEvaluator
from sentence_transformers.util import batch_to_device
from torch import Tensor
from torch.utils.data import DataLoader
from transformers import MT5EncoderModel, AutoConfig
from transformers import pipeline, AutoTokenizer

model_name = "google/mt5-small"
base_model = models.Transformer(model_name)
# sentence bert repo does not current support mT5 yet (only T5 ironically)
MT5EncoderModel._keys_to_ignore_on_load_unexpected = ["decoder.*"]
mT5_config = AutoConfig.from_pretrained(model_name)   
tokenizer = AutoTokenizer.from_pretrained(model_name)
ex=tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

Some weights of the model checkpoint at google/mt5-small were not used when initializing MT5Model: ['lm_head.weight']
- This IS expected if you are initializing MT5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MT5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
import pandas as pd
train_df=pd.read_csv('C:/Users/leose/second_semester/NLP/malawi_news_classification/data/train.csv')
test_df=pd.read_csv('C:/Users/leose/second_semester/NLP/malawi_news_classification/data/test.csv')

In [3]:
train_batch_sentences=[]
test_batch_sentences=[]

for frame in train_df['Text']:
    train_batch_sentences.append(frame)
for sentence in test_df['Text']:
    test_batch_sentences.append(frame)

In [5]:
import pickle
train_representations=tokenizer(train_batch_sentences,padding=True,truncation=True)
test_representations=tokenizer(test_batch_sentences,padding=True,truncation=True)

train_filehandler = open("train.pkl","wb")
test_filehandler = open("test.pkl","wb")
pickle.dump(train_representations,train_filehandler)
pickle.dump(test_representations,test_filehandler)
train_filehandler.close()
test_filehandler.close()

In [13]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder 
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

X_train=train_representations['input_ids'][0:1336]
print(len(X_train))
y_train=train_df['Label'][0:1336]
print(len(y_train))

model = XGBClassifier() 
model.fit(X_train, y_train)


1336
1336
100
[13:39:55] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [17]:
X_test=train_representations['input_ids'][1337:1435]
y_test=train_df['Label'][1337:1435]

y_pred = model.predict(X_test) 
predictions = [value for value in y_pred]
accuracy = accuracy_score(y_test, predictions) 
print(accuracy)

0.20408163265306123
